In [1]:
import os
import json
import pandas as pd

import difflib
import nltk
import re

from rouge_s import py_rouge_scores
from bert_score import score
from IPython.core.display import display, HTML

%matplotlib inline

/tmp/ipykernel_3817389/3962767863.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [77]:
name_method = {1: '1_bart_noprompt', 2: '2_bart_topic', 3: '3_bart_length', 
               4: '4_bart_topic_length', 5: '5_bart_contrastive_random', 6: '6_bart_contrastive_synonym',
               7: '7_bart_contrastive_combine', 8: '8_bart_contrastive_combine_word_tagger_dialog', 9: '9_bart_contrastive_combine_prompt_tagger_dialog',
               10: '5_bart_contrastive_random_negative', 11: '6_bart_contrastive_synonym_positive', 12: '7_bart_contrastive_combine_pos_neg',
               13: '4_bart_topic_length_word_tagger', 14: '4_bart_topic_length_prompt_tagger'}

In [78]:
total_method = 14

list_method = list(range(1 ,total_method + 1))

In [79]:
gold_summary_file = './dataset.xlsx'

In [80]:
gold_summary_df = pd.read_excel(gold_summary_file, index_col=0)

In [81]:
gold_summary_df.head()

,fname,dialogue,summary1,summary2,summary3,topic1,topic2,topic3
0,test_0,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson helps #Person1# to write a memo to ...,In order to prevent employees from wasting tim...,Ms. Dawson takes a dictation for #Person1# abo...,communication method,company policy,dictation
1,test_1,#Person1#: You're finally here! What took so l...,#Person2# arrives late because of traffic jam....,#Person2# decides to follow #Person1#'s sugges...,#Person2# complains to #Person1# about the tra...,public transportation,transportation,discuss transportation
2,test_2,"#Person1#: Kate, you never believe what's happ...",#Person1# tells Kate that Masha and Hero get d...,#Person1# tells Kate that Masha and Hero are g...,#Person1# and Kate talk about the divorce betw...,divorce,divorce,discuss divorce
3,test_3,"#Person1#: Happy Birthday, this is for you, Br...",#Person1# and Brian are at the birthday party ...,#Person1# attends Brian's birthday party. Bria...,#Person1# has a dance with Brian at Brian's bi...,birthday party,birthday party,birthday party
4,test_4,#Person1#: This Olympic park is so big!\n#Pers...,#Person1# is surprised at the Olympic Stadium'...,#Person2# shows #Person1# around the construct...,#Person2# introduces the Olympic Stadium's fin...,Olympic Stadium,sports stadium,Olympic Stadium


In [82]:
result_summary_file = './result.xlsx'

In [83]:
result_all_df = {}
for i in list(range(0 ,total_method)):
    result_df = pd.read_excel(result_summary_file, sheet_name=i, index_col=0)
    result_all_df[i+1] = result_df

In [84]:
result_all_df[1].head()

,fname,gen_summary1,gen_summary2,gen_summary3,gen1_rouge_1,gen1_rouge_2,gen1_rouge_l,gen1_bert_score,gen1_gold_length,gen1_gen_length,...,gen2_gold_length,gen2_gen_length,gen2_diff_length,gen3_rouge_1,gen3_rouge_2,gen3_rouge_l,gen3_bert_score,gen3_gold_length,gen3_gen_length,gen3_diff_length
0,test_0,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...,0.350000,0.051282,0.317362,0.891681,27,53,...,36,53,17,0.425000,0.230769,0.394919,0.884799,27,53,26
1,test_1,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic again. #Person1...,0.400000,0.052632,0.466322,0.910078,22,18,...,18,18,0,0.514286,0.181818,0.466024,0.919479,17,18,1
2,test_2,#Person1# tells Kate that Masha and Hero are g...,#Person1# tells Kate that Masha and Hero are g...,#Person1# tells Kate that Masha and Hero are g...,0.871795,0.702703,0.892000,0.971663,19,20,...,20,20,0,0.634146,0.205128,0.549823,0.936697,21,20,-1
3,test_3,#Person1# celebrates Brian's birthday and danc...,#Person1# celebrates Brian's birthday and danc...,#Person1# celebrates Brian's birthday and danc...,0.342857,0.000000,0.292332,0.903291,18,15,...,12,15,3,0.555556,0.176471,0.343960,0.919318,18,15,-3
4,test_4,#Person1# and #Person2# are in the Olympic sta...,#Person1# and #Person2# are in the Olympic sta...,#Person1# and #Person2# are in the Olympic sta...,0.344828,0.148148,0.411817,0.899916,13,15,...,12,15,3,0.413793,0.148148,0.411817,0.904353,13,15,2


In [85]:
list_gold_summary = ['summary1', 'summary2', 'summary3']
list_gold_topic = ['topic1', 'topic2', 'topic3']
list_gen_summary = ['gen_summary1', 'gen_summary2', 'gen_summary3']

In [86]:
num_example = 7
gen_summary_num = 1

print("Test Number: ", num_example)
print("Gold Topic: ", gold_summary_df.loc[num_example, list_gold_topic[gen_summary_num-1]])
print("="*150)
print("Gold Summary: ", gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]])
print("-"*150)
for num_method in list_method:
    # print("Experiment: ", name_method[num_method])
    
    # print("gen_summary1: ", result_all_df[num_method].loc[num_example, 'gen_summary1'])
    # print("gen_summary2: ", result_all_df[num_method].loc[num_example, 'gen_summary2'])
    # print("gen_summary3: ", result_all_df[num_method].loc[num_example, 'gen_summary3'])
    # print("-"*150)
    
    # print(f"{name_method[num_method]}: ", list_gen_summary[gen_summary_num-1], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print(f"{name_method[num_method]}: ", result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print("-"*150)

Test Number:  7
Gold Topic:  bill
Gold Summary:  #Person2# is checking out and asks #Person1# for the bill. #Person1# gives #Person2# a wrong bill at first then corrects it.
------------------------------------------------------------------------------------------------------------------------------------------------------
1_bart_noprompt:  #Person1# helps #Person2# check out and finds the charge for laundry service on Nov. 20th has been added to someone else's. They'll correct the bill.
------------------------------------------------------------------------------------------------------------------------------------------------------
2_bart_topic:  #Person2# is checking out and finds the bill is wrong. #Person1# will correct it with the department concerned.
------------------------------------------------------------------------------------------------------------------------------------------------------
3_bart_length:  #Person1# helps #Person2# check out and finds the charge for l

In [87]:
num_example = 29
gen_summary_num = 1

print("Test Number: ", num_example)
print("Gold Topic: ", gold_summary_df.loc[num_example, list_gold_topic[gen_summary_num-1]])
print("="*150)
print("Gold Summary: ", gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]])
print("-"*150)
for num_method in list_method:
    # print("Experiment: ", name_method[num_method])
    
    # print("gen_summary1: ", result_all_df[num_method].loc[num_example, 'gen_summary1'])
    # print("gen_summary2: ", result_all_df[num_method].loc[num_example, 'gen_summary2'])
    # print("gen_summary3: ", result_all_df[num_method].loc[num_example, 'gen_summary3'])
    # print("-"*150)
    
    # print(f"{name_method[num_method]}: ", list_gen_summary[gen_summary_num-1], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print(f"{name_method[num_method]}: ", result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print("-"*150)

Test Number:  29
Gold Topic:  communication strategy
Gold Summary:  #Person2# tells #Person1# #Person2#'s communication strategy.
------------------------------------------------------------------------------------------------------------------------------------------------------
1_bart_noprompt:  #Person2# tells #Person1# #Person2 # is an outgoing person and the most important thing in communication is sincerity.
------------------------------------------------------------------------------------------------------------------------------------------------------
2_bart_topic:  #Person2# tells #Person1# #Person2 # is an outgoing person and the most important thing in communication is sincerity.
------------------------------------------------------------------------------------------------------------------------------------------------------
3_bart_length:  #Person2# tells #Person1# about #Person2 #'s personality.
------------------------------------------------------------------------

In [88]:
num_example = 450
gen_summary_num = 3

print("Test Number: ", num_example)
print("Gold Topic: ", gold_summary_df.loc[num_example, list_gold_topic[gen_summary_num-1]])
print("="*150)
print("Gold Summary: ", gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]])
print("-"*150)
for num_method in list_method:
    # print("Experiment: ", name_method[num_method])
    
    # print("gen_summary1: ", result_all_df[num_method].loc[num_example, 'gen_summary1'])
    # print("gen_summary2: ", result_all_df[num_method].loc[num_example, 'gen_summary2'])
    # print("gen_summary3: ", result_all_df[num_method].loc[num_example, 'gen_summary3'])
    # print("-"*150)
    
    # print(f"{name_method[num_method]}: ", list_gen_summary[gen_summary_num-1], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print(f"{name_method[num_method]}: ", result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print("-"*150)

Test Number:  450
Gold Topic:  ask for the way
Gold Summary:  #Person1# asks #Person2# for the way to Peak Tram. #Person2# tells so and is asked to repeat so #Person1# can write it down.
------------------------------------------------------------------------------------------------------------------------------------------------------
1_bart_noprompt:  #Person2# tells #Person1# the way to the Peak Tram and asks #Person 1# to write it down.
------------------------------------------------------------------------------------------------------------------------------------------------------
2_bart_topic:  #Person2# tells #Person1# the way to the Peak Tram.
------------------------------------------------------------------------------------------------------------------------------------------------------
3_bart_length:  #Person2# tells #Person1# the way to the Peak Tram. #Person3# repeats the directions and asks #Person 1# to write it down. # person1# is grateful.
-----------------------

In [89]:
num_example = 307
gen_summary_num = 2

print("Test Number: ", num_example)
print("Gold Topic: ", gold_summary_df.loc[num_example, list_gold_topic[gen_summary_num-1]])
print("="*150)
print("Gold Summary: ", gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]])
print("-"*150)
for num_method in list_method:
    # print("Experiment: ", name_method[num_method])
    
    # print("gen_summary1: ", result_all_df[num_method].loc[num_example, 'gen_summary1'])
    # print("gen_summary2: ", result_all_df[num_method].loc[num_example, 'gen_summary2'])
    # print("gen_summary3: ", result_all_df[num_method].loc[num_example, 'gen_summary3'])
    # print("-"*150)
    
    # print(f"{name_method[num_method]}: ", list_gen_summary[gen_summary_num-1], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print(f"{name_method[num_method]}: ", result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print("-"*150)

Test Number:  307
Gold Topic:  warning
Gold Summary:  #Person1# warns #Person2# of the possible trouble in the future, but #Person2# doesn't mind.
------------------------------------------------------------------------------------------------------------------------------------------------------
1_bart_noprompt:  #Person1# warns #Person2# not to do what they're doing because it may lead to trouble.
------------------------------------------------------------------------------------------------------------------------------------------------------
2_bart_topic:  #Person1# warns #Person2# not to do something if it doesn't cause trouble on the horizon, but #Person 2# keeps looking at the positive side of things.
------------------------------------------------------------------------------------------------------------------------------------------------------
3_bart_length:  #Person1# warns #Person2# not to do something if he thinks everything has gone as smooth as clockwork.
----------

In [90]:
num_example = 176
gen_summary_num = 1

print("Test Number: ", num_example)
print("Gold Topic: ", gold_summary_df.loc[num_example, list_gold_topic[gen_summary_num-1]])
print("="*150)
print("Gold Summary: ", gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]])
print("-"*150)
for num_method in list_method:
    # print("Experiment: ", name_method[num_method])
    
    # print("gen_summary1: ", result_all_df[num_method].loc[num_example, 'gen_summary1'])
    # print("gen_summary2: ", result_all_df[num_method].loc[num_example, 'gen_summary2'])
    # print("gen_summary3: ", result_all_df[num_method].loc[num_example, 'gen_summary3'])
    # print("-"*150)
    
    # print(f"{name_method[num_method]}: ", list_gen_summary[gen_summary_num-1], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print(f"{name_method[num_method]}: ", result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print("-"*150)

Test Number:  176
Gold Topic:  taxi conversation
Gold Summary:  #Person1# drives #Person2# to the Grand Hotel.
------------------------------------------------------------------------------------------------------------------------------------------------------
1_bart_noprompt:  #Person2# shows #Person1# the way to the Grand Hotel.
------------------------------------------------------------------------------------------------------------------------------------------------------
2_bart_topic:  #Person2# tells #Person1# the Grand Hotel is on 11th Avenue, Park Street.
------------------------------------------------------------------------------------------------------------------------------------------------------
3_bart_length:  #Person2# tells #Person1# the location of the Grand Hotel.
------------------------------------------------------------------------------------------------------------------------------------------------------
4_bart_topic_length:  #Person2# tells #Person1# t

In [92]:
num_example = 173
gen_summary_num = 1

print("Test Number: ", num_example)
print("Gold Topic: ", gold_summary_df.loc[num_example, list_gold_topic[gen_summary_num-1]])
print("="*150)
print("Gold Summary: ", gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]])
print("-"*150)
for num_method in list_method:
    # print("Experiment: ", name_method[num_method])
    
    # print("gen_summary1: ", result_all_df[num_method].loc[num_example, 'gen_summary1'])
    # print("gen_summary2: ", result_all_df[num_method].loc[num_example, 'gen_summary2'])
    # print("gen_summary3: ", result_all_df[num_method].loc[num_example, 'gen_summary3'])
    # print("-"*150)
    
    # print(f"{name_method[num_method]}: ", list_gen_summary[gen_summary_num-1], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print(f"{name_method[num_method]}: ", result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print("-"*150)

Test Number:  173
Gold Topic:  ordering food
Gold Summary:  #Person2# orders a glass of lemonade, barbeque wings, and baby-back ribs for meal with #Person1#'s assistance.
------------------------------------------------------------------------------------------------------------------------------------------------------
1_bart_noprompt:  #Person1# helps #Person2# order a glass of lemonade, barbeque wings, and baby-back ribs.
------------------------------------------------------------------------------------------------------------------------------------------------------
2_bart_topic:  #Person1# helps #Person2# order a glass of lemonade, barbeque wings, and baby-back ribs.
------------------------------------------------------------------------------------------------------------------------------------------------------
3_bart_length:  #Person1# helps #Person2# order a glass of lemonade, barbeque wings, and baby-back ribs.
------------------------------------------------------------

In [111]:
num_example = 226
gen_summary_num = 3

print("Test Number: ", num_example)
print("Gold Topic: ", gold_summary_df.loc[num_example, list_gold_topic[gen_summary_num-1]])
print("="*150)
print("Gold Summary: ", gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]])
print("-"*150)
for num_method in list_method:
    # print("Experiment: ", name_method[num_method])
    
    # print("gen_summary1: ", result_all_df[num_method].loc[num_example, 'gen_summary1'])
    # print("gen_summary2: ", result_all_df[num_method].loc[num_example, 'gen_summary2'])
    # print("gen_summary3: ", result_all_df[num_method].loc[num_example, 'gen_summary3'])
    # print("-"*150)
    
    # print(f"{name_method[num_method]}: ", list_gen_summary[gen_summary_num-1], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print(f"{name_method[num_method]}: ", result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]])
    print("-"*150)

Test Number:  226
Gold Topic:  leave a tip
Gold Summary:  #Person1# pays and leaves a tip but #Person2# politely refuses.
------------------------------------------------------------------------------------------------------------------------------------------------------
1_bart_noprompt:  #Person2# helps #Person1# order food and tells him they don't take tips.
------------------------------------------------------------------------------------------------------------------------------------------------------
2_bart_topic:  #Person1# pays for his meal and wants to leave a tip. #Person2# says they don't take tips.
------------------------------------------------------------------------------------------------------------------------------------------------------
3_bart_length:  #Person2# helps #Person1# pay for his meal and tells him there's no tip.
-------------------------------------------------------------------------------------------------------------------------------------------

In [91]:
'''
Find the matching substrings in 2 strings.
:parameter
    :param a: string - raw text
    :param b: string - raw text
:return
    2 lists used in to display matches
'''
def utils_split_sentences(a, b):
    ## find clean matches
    match = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=True)
    lst_match = [block for block in match.get_matching_blocks() if block.size > 20]

    ## difflib didn't find any match
    if len(lst_match) == 0:
        lst_a, lst_b = nltk.sent_tokenize(a), nltk.sent_tokenize(b)

    ## work with matches
    else:
        first_m, last_m = lst_match[0], lst_match[-1]

        ### a
        string = a[0 : first_m.a]
        lst_a = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = a[m.a : m.a+m.size]
            lst_a.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = a[m.a+m.size : next_m.a]
                lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = a[last_m.a+last_m.size :]
        lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]

        ### b
        string = b[0 : first_m.b]
        lst_b = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = b[m.b : m.b+m.size]
            lst_b.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = b[m.b+m.size : next_m.b]
                lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = b[last_m.b+last_m.size :]
        lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]

    return lst_a, lst_b


'''
Highlights the matched strings in text.
:parameter
    :param a: string - raw text
    :param b: string - raw text
    :param both: bool - search a in b and, if True, viceversa
    :param sentences: bool - if False matches single words
:return
    text html, it can be visualized on notebook with display(HTML(text))
'''
def display_string_matching(a, b, both=True, sentences=True, titles=[]):
    if sentences is True:
        lst_a, lst_b = utils_split_sentences(a, b)
    else:
        lst_a, lst_b = a.split(), b.split()

    ## highlight a
    first_text = []
    for i in lst_a:
        if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_b]:
            first_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
        else:
            first_text.append(i)
    first_text = ' '.join(first_text)

    ## highlight b
    second_text = []
    if both is True:
        for i in lst_b:
            if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_a]:
                second_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
            else:
                second_text.append(i)
    else:
        second_text.append(b)
    second_text = ' '.join(second_text)

    ## concatenate
    if len(titles) > 0:
        first_text = "<strong>"+titles[0]+"</strong><br>"+first_text
    if len(titles) > 1:
        second_text = "<strong>"+titles[1]+"</strong><br>"+second_text
    else:
        second_text = "---"*65+"<br><br>"+second_text
    final_text = first_text +'<br>'+ second_text
    return final_text

In [28]:
num_example = 7
gen_summary_num = 1

In [29]:
for num_method in list_method:
    print("-"*100)
    print("Experiment: ", name_method[num_method])
    match = display_string_matching(gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]], both=True, sentences=False, titles=["Full Text", "Actual Text"])
    display(HTML(match))

----------------------------------------------------------------------------------------------------
Experiment:  1_bart_noprompt


----------------------------------------------------------------------------------------------------
Experiment:  2_bart_topic


----------------------------------------------------------------------------------------------------
Experiment:  3_bart_length


----------------------------------------------------------------------------------------------------
Experiment:  4_bart_topic_length


----------------------------------------------------------------------------------------------------
Experiment:  5_bart_contrastive_random


----------------------------------------------------------------------------------------------------
Experiment:  6_bart_contrastive_synonym


----------------------------------------------------------------------------------------------------
Experiment:  7_bart_contrastive_combine


----------------------------------------------------------------------------------------------------
Experiment:  8_bart_contrastive_combine_word_tagger_dialog


----------------------------------------------------------------------------------------------------
Experiment:  9_bart_contrastive_combine_prompt_tagger_dialog


In [30]:
num_method = 4

for gen_summary_num in range(3):
    print("-"*100)
    print(f"Generated summary {gen_summary_num}:")
    match = display_string_matching(gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]], both=True, sentences=False, titles=["Full Text", "Actual Text"])
    display(HTML(match))

----------------------------------------------------------------------------------------------------
Generated summary 0:


----------------------------------------------------------------------------------------------------
Generated summary 1:


----------------------------------------------------------------------------------------------------
Generated summary 2:


In [31]:
# num_example = 7
# gen_summary_num = 1
num_method = 4

print("Experiment: ", name_method[num_method])
match = display_string_matching(gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]], both=True, sentences=False, titles=["Full Text", "Actual Text"])
display(HTML(match))

Experiment:  4_bart_topic_length


In [32]:
# num_example = 7
# gen_summary_num = 1
num_method = 7

print("Experiment: ", name_method[num_method])
match = display_string_matching(gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]], both=True, sentences=False, titles=["Full Text", "Actual Text"])
display(HTML(match))

Experiment:  7_bart_contrastive_combine


In [33]:
# num_example = 7
gen_summary_num = 1
num_method = 7

print("Experiment: ", name_method[num_method])
match = display_string_matching(gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]], both=True, sentences=False, titles=["Full Text", "Actual Text"])
display(HTML(match))

Experiment:  7_bart_contrastive_combine


In [34]:
# num_example = 7
gen_summary_num = 2
# num_method = 4

print("Experiment: ", name_method[num_method])
match = display_string_matching(gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]], both=True, sentences=False, titles=["Full Text", "Actual Text"])
display(HTML(match))

Experiment:  7_bart_contrastive_combine


In [ ]:
# num_example = 7
gen_summary_num = 3
# num_method = 4

print("Experiment: ", name_method[num_method])
match = display_string_matching(gold_summary_df.loc[num_example, list_gold_summary[gen_summary_num-1]], result_all_df[num_method].loc[num_example, list_gen_summary[gen_summary_num-1]], both=True, sentences=False, titles=["Full Text", "Actual Text"])
display(HTML(match))